In [1]:
!wget https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip

--2021-05-05 14:37:59--  https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20194537 (19M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  19.26M  53.5MB/s    in 0.4s    

2021-05-05 14:38:00 (53.5 MB/s) - ‘data.zip’ saved [20194537/20194537]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/cats/
  inflating: data/cats/cat.1.jpg     
  inflating: data/cats/cat.10.jpg    
  inflating: data/cats/cat.100.jpg   
  inflating: data/cats/cat.101.jpg   
  inflating: data/cats/cat.102.jpg   
  inflating: data/cats/cat.103.jpg   
  inflating: data/cats/cat.104.jpg   
  inflating: data/cats/cat.105.jpg   
  inflating: data/cats/cat.106.jpg   
  inflating: data/cats/cat.107.jpg   
  inflating: data/cats/cat.108.jpg   
  inflating: data/cats/cat.109.jpg   
  inflating: data/cats/cat.11.jpg    
  inflating: data/cats/cat.110.jpg   
  inflating: data/cats/cat.111.jpg   
  inflating: data/cats/cat.112.jpg   
  inflating: data/cats/cat.113.jpg   
  inflating: data/cats/cat.114.jpg   
  inflating: data/cats/cat.115.jpg   
  inflating: data/cats/cat.116.jpg   
  inflating: data/cats/cat.117.jpg   
  inflating: data/cats/cat.118.jpg   
  inflating: data/cats/cat.119.jpg   
  inflating: data/cats/cat.12.jpg    
  inflating: data/cats/cat

In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
print(len(os.listdir('./data/Humans/')))
print(len(os.listdir('./data/cats/')))
print(len(os.listdir('./data/dogs/')))
print(len(os.listdir('./data/horses/')))

202
202
202
202


In [5]:
path_humans = './data/Humans'
path_cats = './data/cats'
path_dogs = './data/dogs/'
path_horses = './data/horses/'

humans = os.listdir(path_humans)
cats = os.listdir(path_cats)
dogs = os.listdir(path_dogs)
horses = os.listdir(path_horses)

human_img_sizes = []
cats_img_sizes = []
dogs_img_sizes = []
horses_img_sizes = []

for i in range(202):
    hu = plt.imread(os.path.join(path_humans, humans[i]))
    c = plt.imread(os.path.join(path_cats, cats[i]))
    d = plt.imread(os.path.join(path_dogs, dogs[i]))
    ho = plt.imread(os.path.join(path_horses, horses[i]))
    human_img_sizes.append(hu.shape[:-1])
    cats_img_sizes.append(c.shape[:-1])
    dogs_img_sizes.append(d.shape[:-1])
    horses_img_sizes.append(ho.shape[:-1])


In [6]:
mean_sizes = [np.mean(human_img_sizes, axis=0), np.mean(cats_img_sizes, axis=0),
              np.mean(dogs_img_sizes, axis=0), np.mean(horses_img_sizes, axis=0)]

target_size = np.array(mean_sizes).mean(axis=0)
target_size

array([304.6200495 , 329.92574257])

In [7]:
image_gen = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

In [8]:
train = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')

val = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

Found 728 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [17]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

img_size = [304,329,3]

In [18]:
input = Input(shape = img_size)
model = ResNet50(input_tensor=input,)

In [19]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [24]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
    layer.trainable = False


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")

In [26]:
my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [27]:
model.fit(train, validation_data=val, epochs=50, callbacks=my_callbacks)

Epoch 1/50
182/182 [==============================] - 285s 2s/step - loss: 1.5015 - accuracy: 0.2700 - val_loss: 1.3525 - val_accuracy: 0.3625
Epoch 2/50
182/182 [==============================] - 280s 2s/step - loss: 1.3298 - accuracy: 0.3768 - val_loss: 1.3158 - val_accuracy: 0.2875
Epoch 3/50
182/182 [==============================] - 280s 2s/step - loss: 1.3425 - accuracy: 0.3678 - val_loss: 1.3493 - val_accuracy: 0.3875
Epoch 4/50
182/182 [==============================] - 279s 2s/step - loss: 1.3341 - accuracy: 0.3532 - val_loss: 1.3421 - val_accuracy: 0.2750
Epoch 5/50
182/182 [==============================] - 278s 2s/step - loss: 1.3425 - accuracy: 0.3775 - val_loss: 1.2884 - val_accuracy: 0.3375
Epoch 6/50
182/182 [==============================] - 277s 2s/step - loss: 1.2362 - accuracy: 0.3856 - val_loss: 1.1601 - val_accuracy: 0.4375
Epoch 7/50
182/182 [==============================] - 276s 2s/step - loss: 1.2712 - accuracy: 0.4108 - val_loss: 1.2393 - val_accuracy: 0.3625

In [28]:
model.history.history

{'accuracy': [0.2898351550102234,
  0.34065935015678406,
  0.3681318759918213,
  0.34752747416496277,
  0.3928571343421936,
  0.3997252881526947,
  0.40521979331970215,
  0.45879119634628296,
  0.45467033982276917,
  0.4642857015132904,
  0.4423076808452606,
  0.45604395866394043,
  0.48351648449897766,
  0.4821428656578064,
  0.450549453496933,
  0.48763737082481384,
  0.49038460850715637,
  0.5027472376823425,
  0.5013736486434937,
  0.4780219793319702,
  0.5123626589775085,
  0.5123626589775085,
  0.49175822734832764,
  0.5151098966598511,
  0.5054945349693298],
 'loss': [1.4420756101608276,
  1.335198998451233,
  1.3590012788772583,
  1.3081705570220947,
  1.314832329750061,
  1.2460980415344238,
  1.2693768739700317,
  1.1817951202392578,
  1.230064868927002,
  1.1977522373199463,
  1.1861248016357422,
  1.2109997272491455,
  1.152764916419983,
  1.145262360572815,
  1.1766737699508667,
  1.1526199579238892,
  1.1309022903442383,
  1.1133519411087036,
  1.133056879043579,
  1.1247